In [ ]:
!pip install git+https://github.com/neulab/BARTScore.git

In [ ]:
!git clone https://github.com/neulab/BARTScore.git
%cd BARTScore

!pip install -q transformers==4.36.2 torch torchvision torchaudio pandas sentencepiece

In [ ]:
# ==============================================
# BARTScore Evaluation Script
# ==============================================

# --- Installation & Setup ---
!git clone https://github.com/neulab/BARTScore.git
%cd BARTScore

!pip install -q transformers==4.36.2 torch torchvision torchaudio pandas sentencepiece

import torch
import pandas as pd
import sys
sys.path.append('.')

from bart_score import BARTScorer

# --- Device Setup ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

bart_scorer = BARTScorer(device=device, checkpoint='facebook/bart-large-cnn')

# --- Data (insert your own lists) ---
candidates = [ " Here paste the 80 generated explanations each seperated by comma. "
]

references = [ " Here paste the 80 reference explanations each seperated by comma. "
]

# Safety check
if len(candidates) != len(references):
    raise ValueError("candidates and references must have the SAME length!")

# --- Compute BARTScore ---
print("Computing BARTScores...")
scores = bart_scorer.score(candidates, references, batch_size=8)

avg_score = sum(scores) / len(scores)
print(f"\n Average BARTScore: {avg_score:.4f}")

# --- Prepare DataFrame for Output ---
df = pd.DataFrame({
    "Example_ID": [f"E{i+1}" for i in range(len(candidates))],
    "Generated_NL": candidates,
    "Ground_Truth_NL": references,
    "BARTScore": scores
})

# Add average row
df.loc[len(df)] = ["Average", "", "", avg_score]

# --- Save Results ---
output_path = "/content/BARTScore_MXPX.csv"
df.to_csv(output_path, index=False)

print(f"BARTScore evaluation complete.")
print(f"Results saved to: {output_path}\n")

# --- Print First Few Scores ---
for i in range(min(5, len(scores))):
    print(f"Example {i+1}: BARTScore = {scores[i]:.4f}")